In [21]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import sys
import numpy as np

files = ['cat1.png']

net = load_model('model.h5')

cls_list = ['cat', 'dog']

for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    initial_class = np.argmax(pred)

    print(f,' is ',cls_list[top_inds[0]])
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))
    print()

cat1.png  is  cat
    1.000  cat
    0.000  dog



In [22]:

# Get current session (assuming tf backend)
sess = K.get_session()
# Initialize adversarial example with input image
x_adv = x
# Added noise
x_noise = np.zeros_like(x)

In [23]:
epochs = 50
epsilon = 0.01
prev_probs = []

for i in range(epochs): 
    # One hot encode the initial class
    target = K.one_hot(initial_class, 2)
    
    # Get the loss and gradient of the loss wrt the inputs
    loss = K.categorical_crossentropy(target, net.output)
    grads = K.gradients(loss, net.input)

    # Get the sign of the gradient
    delta = K.sign(grads[0])
    #x_noise = x_noise + delta

    # Perturb the image
    x_adv = x_adv + epsilon*delta

    # Get the new image and predictions
    x_adv = sess.run(x_adv, feed_dict={net.input:x})
    preds = net.predict(x_adv)

    # Store the probability of the target class
    prev_probs.append(preds[0][initial_class])
    
    print(i)
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [24]:
pred = net.predict(x_adv)[0]
top_inds = pred.argsort()[::-1][:5]


print(f,' is ',cls_list[top_inds[0]])
for i in top_inds:
    print('    {:.3f}  {}'.format(pred[i], cls_list[i]))
print()

cat1.png  is  dog
    0.994  dog
    0.006  cat



In [25]:
image.array_to_img(np.squeeze(x_adv)).save('ad_cat1.png')